In [2]:
from matplotlib import pyplot as plt
import os
import sqlite3

DB_PATH = 'CoronaBotDB'
con = sqlite3.connect(DB_PATH+"/newkorea.db", isolation_level=None)
con = con.cursor()
con.execute("SELECT * FROM region")

item_list = []
for row in con.execute('SELECT * FROM region'):
    item_list.append({'date': row[0], 'region': row[1], 'con':int(row[2])})
    
print(item_list)

[{'date': '2020년 12월 16일 00시', 'region': '합계', 'con': 1078}, {'date': '2020년 12월 16일 00시', 'region': '서울', 'con': 378}, {'date': '2020년 12월 16일 00시', 'region': '경기', 'con': 329}, {'date': '2020년 12월 16일 00시', 'region': '전북', 'con': 75}, {'date': '2020년 12월 16일 00시', 'region': '인천', 'con': 67}, {'date': '2020년 12월 16일 00시', 'region': '부산', 'con': 41}, {'date': '2020년 12월 16일 00시', 'region': '충남', 'con': 35}, {'date': '2020년 12월 16일 00시', 'region': '경북', 'con': 28}, {'date': '2020년 12월 16일 00시', 'region': '대구', 'con': 27}, {'date': '2020년 12월 16일 00시', 'region': '충북', 'con': 23}, {'date': '2020년 12월 16일 00시', 'region': '경남', 'con': 19}, {'date': '2020년 12월 16일 00시', 'region': '제주', 'con': 15}, {'date': '2020년 12월 16일 00시', 'region': '대전', 'con': 15}, {'date': '2020년 12월 16일 00시', 'region': '강원', 'con': 8}, {'date': '2020년 12월 16일 00시', 'region': '울산', 'con': 6}, {'date': '2020년 12월 16일 00시', 'region': '검역', 'con': 6}, {'date': '2020년 12월 16일 00시', 'region': '전남', 'con': 4}, {'date': '202

In [3]:
import numpy as np
import pandas as pd
import geopandas as gpd
from geopandas.tools import geocode
import math
from collections import namedtuple

import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster, TimestampedGeoJson

import datetime
import json
from folium.features import DivIcon

ModuleNotFoundError: No module named 'geopandas'

In [4]:
#!pip install geopandas

    ERROR: Command errored out with exit status 1:
     command: 'c:\users\82108\anaconda3\python.exe' -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\82108\\AppData\\Local\\Temp\\pip-install-b325gpwc\\fiona\\setup.py'"'"'; __file__='"'"'C:\\Users\\82108\\AppData\\Local\\Temp\\pip-install-b325gpwc\\fiona\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\82108\AppData\Local\Temp\pip-pip-egg-info-qo92urs0'
         cwd: C:\Users\82108\AppData\Local\Temp\pip-install-b325gpwc\fiona\
    Complete output (1 lines):
    A GDAL API version must be specified. Provide a path to gdal-config using a GDAL_CONFIG environment variable or use a GDAL_VERSION environment variable.
    ----------------------------------------
ERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command out

In [5]:
def create_geojson_features(df_con,df_on,
                        radius_max=1000,
                        radius_min = 2, 
                        fill_color_confirmed = '#FC766AFF',
                        fill_color_recovered = '#0A5E2AFF',
                        fill_color_death = '#E80018',
                        weight = 1,
                        fill_opacity = 0.5
                        ):

print('> Creating GeoJSON features...')

features = []
feature = []
# df_con
for _, row in df_con.iterrows():
    radius = np.sqrt(row['Confirmed'])
    if radius != 0:
      if radius < radius_min:
        radius = radius_min

      if radius > radius_max:
        radius = radius_max

      popup = str(row['city']) + str(row['Confirmed'])

      feature = {
          'type': 'Feature',
          'geometry': {
              'type':'Point', 
              'coordinates':[row['longitude'],row['latitude']]
          },
          'properties': {
              'time': row['date'].__str__(),
              'style': {'color' : fill_color_confirmed},
              'icon': 'circle',
              'iconstyle':{
                  'fillColor': fill_color_confirmed,
                  'fillOpacity': fill_opacity,
                  'stroke': 'true',
                  'radius': radius,
                  'weight': weight,
                  'popup': popup
              }
          }
    }
    features.append(feature)

for _, row in df_on.iterrows():
    radius = np.sqrt(row['Confirmed'])
    if radius != 0:
      if radius < radius_min:
        radius = radius_min

      if radius > radius_max:
        radius = radius_max
    popups = str(row['city']) + str(row['Confirmed'])
    size=radius,radius
    feature = {
        'type': 'Feature',
        'geometry': {
            'type':'Point',
            'coordinates':[row['longitude'],row['latitude']]
        },
        'properties': {
            'time': row['date'].__str__(),
            'style': {'color' : fill_color_confirmed},
            'icon': 'marker',
            'iconstyle': { 'iconUrl': 'https://cdn.iconscout.com/icon/premium/png-512-thumb/coronavirus-4-613136.png', 
                          'iconSize': [radius, radius],
                          'fillOpacity': 0.1,
                          'Popup': popups
                         }
        }
    }
    features.append(feature)



print('> finishing GeoJSON features...')
return features

In [6]:
def make_map(features, caption):
    print('> Making map...')
    coords=[36, 127]
    map = folium.Map(location=coords, 
                               control_scale=True, 
                               zoom_start=6.5,
                               tiles='cartodbpositron',  
                               detect_retina = True
                              )
    
    folium.Choropleth(
        geo_data=kr_geo_data,
        name='choropleth',
        key_on='feature.properties.name',
        fill_color='yellow',
        fill_opacity=0.15,
        line_opacity=0.7
        ).add_to(map)


    TimestampedGeoJson(
        {'type': 'FeatureCollection',
        'features': features}
        , period='P1D'
        , duration='P1D'
        , add_last_point=True
        , auto_play=False
        , loop=False
        , max_speed=1
        , loop_button=True
        , date_options='MM/DD/YYYY'
        , time_slider_drag_update=True
        , transition_time = 500
    ).add_to(map)
  
    
    map.caption = caption
    print('> Done.')
    map.save(os.path.join('Corona_kr.html'))
    
    return map